# Workshop: From Simple to Agentic RAG

<a target="_blank" href="https://colab.research.google.com/github/parambharat/workshops/blob/main/rag/workshop.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<!--- @wandbcode{pc-rag-workshop-nyc} -->

## Setup

Ensure you have a W&B account and API key. You should be able to get one from [here](https://wandb.ai/authorize).

Next, you need to get the Pinecone API key. You can get one from [here](https://app.pinecone.io/settings/keys).

Next head over to [Tavily](https://app.tavily.com/home) and sign up for a free account and get your API key.

We will use [`uv`](https://docs.astral.sh/uv/getting-started/installation/) to install the workshop package and its dependencies.


```bash
!git clone https://github.com/parambharat/workshops.git
%cd workshops/rag
!pip install uv
!uv sync
```

Create an `.env` file in the workshop/rag directory and add the API keys to it or set it in the colab secrets:

```.env
PINECONE_API_KEY="YOUR_PINECONE_API_KEY"
OPENAI_API_KEY="YOUR_OPENAI_API_KEY"
TAVILY_API_KEY="YOUR_TAVILY_API_KEY"
WANDB_API_KEY="YOUR_WANDB_API_KEY"
WANDB_PROJECT="rag-workshop-pc-nyc"
```


In [1]:
IN_COLAB = False
try:
    import google.colab
    IN_COLAB = True
    !git clone https://github.com/ash0ts/workshops.git
    %cd workshops/rag
    !pip install -qqq uv 
    !uv pip install -r pyproject.toml --system -qqq
except:
    pass


### IMPORTANT: If in Colab, restart the kernel

**Restart the kernel to apply the changes from the installed dependencies.**

**Do not run the next cell until you have restarted the kernel.**

**Ensure you don't run the previous cell again after restarting the kernel.**


In [2]:
from set_env import set_env
import os
set_env("PINECONE_API_KEY")
set_env("OPENAI_API_KEY")
set_env("TAVILY_API_KEY")
set_env("WANDB_API_KEY")
set_env("WANDB_PROJECT")

if os.environ.get("WANDB_PROJECT") is None:
    print("WANDB_PROJECT is not set. Setting it to rag-workshop-pc-nyc")
    os.environ["WANDB_PROJECT"] = "rag-workshop-pc-nyc-3"


2025-01-22 15:36:48.214 | WARNING  | set_env.set_env:set_env:109 - 
        Unable to set WANDB_PROJECT=WANDB_PROJECT,
        not in colab or Secrets not set, not kaggle
        or Secrets not set, no .env/dotenv/env file
        in the current working dir or parent dirs.


loading envfile='/Users/anishshah/Documents/Manual Library/GitHub/workshops/rag/.env' with dotenv_values(envfile)
WANDB_PROJECT is not set. Setting it to rag-workshop-pc-nyc


In [3]:
# %load_ext dotenv
# %dotenv
# %load_ext autoreload
# %autoreload 2

In [4]:
import nltk
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("punkt_tab")

!wandb login

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anishshah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/anishshah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/anishshah/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


wandb: Currently logged in as: a-sh0ts. Use `wandb login --relogin` to force relogin


In [5]:
import json
import wandb
import weave
from copy import deepcopy
import nest_asyncio
import pathlib
nest_asyncio.apply()

In [6]:
from utils import convert_contents_to_text, make_id, render_doc, printmd, chunk_simple, run_llm
from retrieval_metrics import RetrievalScorer
from response_metrics import ResponseScorer
from retriever import TfidfSearchEngine, BM25SearchEngine, DenseSearchEngine, Retriever, RetrieverWithReranker, HybridRetrieverWithReranker, VectorStoreSearchEngine
from generation import SimpleResponseGenerator, QueryEnhancedResponseGenerator
from pipeline import SimpleRAGPipeline, QueryEnhancedRAGPipeline
from query_enhancer import QueryEnhancer

/Users/anishshah/Documents/Manual Library/GitHub/workshops/rag/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
Device set to use mps
Device set to use mps
Device set to use mps
Device set to use mps


In [7]:
# WORKSHOP_DATASET = "financebench"
WORKSHOP_DATASET = "sec-10-q"
SAVE_DATA = False
LOAD_PRECOMPUTED_DATA = True
STORE_INTO_PINECONE = False


## Weave

Weave is a lightweight toolkit for tracking and evaluating LLM applications ( This is a very simplified description of Weave though).

- Log and debug language model inputs, outputs, and traces
- Build rigorous, apples-to-apples evaluations for language model use cases
- Organize all the information generated across the LLM workflow, from experimentation to evaluations to production

Don't worry about the details for now. We will see more of it as we build our RAG pipeline.

For now let's initialize W&B Weave. Once intialized, weave will start tracking (more on it later) the inputs and the outputs along with underlying attributes (model name, top_k, etc.) of all the LLMs, functions, evaluations, etc. we will be calling.

In [8]:
# Initialize Weave with the project name
WANDB_PROJECT = os.environ.get("WANDB_PROJECT")
weave_client = weave.init(WANDB_PROJECT)

Logged in as Weights & Biases user: a-sh0ts.
View Weave data at https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/weave


In [9]:
response = await run_llm(messages=[{"role": "user", "content": "What is Weights and Biases and what is Pinecone? Why are they the best fit together?"}])
printmd(response)

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbb-8ed0-7b62-9345-364104d8e120


**Weights and Biases (WandB)** and **Pinecone** are two powerful tools commonly used in the machine learning and data science ecosystem, each serving distinct but complementary purposes.

### Weights and Biases (WandB)

Weights and Biases is a platform designed for experiment tracking, model management, and collaboration in machine learning projects. It provides tools for:

- **Experiment Tracking**: Automatically log hyperparameters, metrics, and system performance during model training.
- **Visualization**: Create visualizations of training runs, compare different models, and analyze performance metrics over time.
- **Collaboration**: Share results and insights with team members, making it easier to collaborate on machine learning projects.
- **Model Management**: Version control for models, datasets, and experiments, allowing teams to keep track of changes and reproduce results.

### Pinecone

Pinecone is a managed vector database designed for machine learning applications, particularly for handling and querying high-dimensional vector embeddings. It provides:

- **Vector Storage**: Efficiently store and manage large volumes of vector embeddings generated by machine learning models.
- **Similarity Search**: Perform fast and scalable similarity searches to find the nearest neighbors of a given vector, which is essential for applications like recommendation systems, image retrieval, and natural language processing.
- **Scalability**: Automatically scales to handle large datasets and high query volumes without the need for complex infrastructure management.

### Why They Are a Good Fit Together

1. **End-to-End Workflow**: Weights and Biases can be used to track experiments and model performance, while Pinecone can be used to deploy and manage the resulting models and their embeddings. This creates a seamless workflow from experimentation to deployment.

2. **Model Evaluation**: After training models with WandB, you can generate embeddings and store them in Pinecone for efficient retrieval and similarity searches. This allows for quick evaluation of model performance in real-world applications.

3. **Collaboration and Sharing**: Teams can use WandB to share insights and results from their experiments, while Pinecone can be used to share and manage the embeddings and similarity search capabilities, facilitating collaboration on projects that require real-time data retrieval.

4. **Scalability**: Both tools are designed to handle large-scale machine learning projects. WandB helps manage the complexity of experiments, while Pinecone provides a scalable solution for managing vector data, making them suitable for enterprise-level applications.

In summary, Weights and Biases and Pinecone complement each other by providing a comprehensive solution for managing the lifecycle of machine learning models, from experimentation to deployment and real-time inference.

## Preparing the dataset
We'll use [Weave Datasets](https://weave-docs.wandb.ai/guides/core-types/datasets) to track and version data as the inputs and outputs of our W&B Runs. 

To learn more about how we prepared the dataset for this workshop, checkout the [dataset preparation script](./download_finance_docs.py).


In [10]:
from download_finance_docs import PDFProcessor

if not LOAD_PRECOMPUTED_DATA:
    if WORKSHOP_DATASET == "financebench":
        kwargs = {
            "REPO_OWNER": "patronus-ai",
            "REPO_NAME": "financebench",
            "DOCS_PATH": "pdfs",
            "data_dir": pathlib.Path("./data/financebench_docs")
        }
    elif WORKSHOP_DATASET == "sec-10-q":
        kwargs = {
            "REPO_OWNER": "docugami",
            "REPO_NAME": "KG-RAG-datasets",
            "DOCS_PATH": "sec-10-q/data/v1/docs",
            "data_dir": pathlib.Path("./data/sec-10-q")
        }
    else:
        raise ValueError(f"Invalid dataset: {WORKSHOP_DATASET}")
    processor = PDFProcessor(**kwargs)
    data = processor.load_pdf_documents()
else:
    if WORKSHOP_DATASET == "financebench":
        data = [dict(datum) for datum in weave.ref("weave:///a-sh0ts/rag-workshop-pc-nyc-financebench/object/raw_data:z3g6P8R5iaLV4EUrAHmGFA13KCtkaXGvLlYNviOr5ZI").get().rows]
    elif WORKSHOP_DATASET == "sec-10-q":
        data = [dict(datum) for datum in weave.ref("weave:///a-sh0ts/rag-workshop-pc-nyc/object/raw_data:bcreeHzxeWoJVBgU4vNORfZVY8HiYRH2zfbTI5HpyOg").get().rows]
    else:
        raise ValueError(f"Invalid dataset: {WORKSHOP_DATASET}")

In [11]:
if not LOAD_PRECOMPUTED_DATA:
    docs_dir = processor.data_dir

In [12]:
if not LOAD_PRECOMPUTED_DATA:
    docs_files = sorted(docs_dir.rglob("*.pdf"))

    print(f"Number of files: {len(docs_files)}\n")
    print("First 5 files:\n{files}".format(files="\n".join(map(str, docs_files[:5]))))

Our documents can be stored as dictionaries with content (_raw text_) and additional metadata.

Metadata is extra information for that data point which can be used to group together similar data points, or filter out a few data points.
We will see in future chapters the importance of metadata and why it should not be ignored while building the ingestion pipeline.

The metadata can be derived (`file_type`) or is inherent (`uri`) to the data point.


In [13]:
total_tokens = sum(map(lambda x: x["metadata"]["raw_tokens"], data))
print(f"Total tokens: {total_tokens}")

Total tokens: 476354



Checking the total number of tokens of your data source is a good practice. In this case, the total tokens is 450k+. Surely, most LLM providers cannot process this many tokens. Building a RAG is justified in such cases.

**Note** We are simply counting words  and calling them `tokens` here. It's only an approximation of the actual token count which will be a bigger number.
In practice we would be using a [tokenizer](https://docs.cohere.com/docs/tokens-and-tokenizers) to calculate the token counts but this naive calculation is an okay approximation for now.

In [14]:
# build weave dataset
if not LOAD_PRECOMPUTED_DATA:
    raw_data = weave.Dataset(name="raw_data", rows=data)

    if SAVE_DATA:
        # publish the dataset
        weave.publish(raw_data)

## A simple RAG pipeline

First, we will build a very simple RAG pipeline.
We will mainly focus on how to preprocess and chunk the data followed by building the simplest retrieval engine without using any fancy "Vector databases".
The idea is to get a sense of the inner workings of a retrieval pipeline and understand the workflow from a user query to a generated response from an LLM.
This end to end workflow will help you understand the importance of each step in a RAG pipeline.

![](./imgs/SimpleRAG.png)

### Chunking the data

Each document contains a large number of tokens, so we need to split it into smaller chunks to manage the number of tokens per chunk. This approach serves three main purposes:

* Most embedding models have a limit of tokens per input (based on their training data and parameters).

* Chunking allows us to retrieve and send only the most relevant portions to our LLM, significantly reducing the total token count. This helps keep the LLM's cost and processing time manageable.

* When the text is small-sized, embedding models tend to generate better vectors as they can capture more fine-grained details and nuances in the text, resulting in more accurate representations.

![](./imgs/SimpleChunking.png)


When choosing chunk size, consider these trade-offs:

- Smaller chunks (100-200 tokens):
  * More precise retrieval
  * Better for finding specific details
  * May lack broader context

- Larger chunks (500-1000 tokens):
  * Provide more context
  * Capture more coherent ideas
  * May introduce noise and reduce precision

The optimal size depends on your data, expected queries, and model capabilities. Experiment with different sizes to find the best balance for your use case.

Here we are chunking each content (text) to a maximum length of 500 tokens (`CHUNK_SIZE`). This is called **FIXED CHUNKING**. Although naive, it's a good starting point.

For now, we will not be overlapping (`CHUNK_OVERLAP`) the content of one chunk with another chunk.

We will be using the `chunk_simple` function to chunk the data.



In [15]:
if LOAD_PRECOMPUTED_DATA:
    if WORKSHOP_DATASET == "financebench":
        document_chunks = [dict(document_chunk) for document_chunk in weave.ref("weave:///a-sh0ts/rag-workshop-pc-nyc-financebench/object/chunked_data:7O0kS8Idu6R8am6Jd7eqkWQ1mookqeNqhit10XsiLFs").get().rows]
    elif WORKSHOP_DATASET == "sec-10-q":
        document_chunks = [dict(document_chunk) for document_chunk in weave.ref("weave:///a-sh0ts/rag-workshop-pc-nyc/object/chunked_data:IkbgNRJ6T8ZLkJD8a9ocbpHxW2f1BThEqY1lagKispc").get().rows]
    else:
        raise ValueError(f"Invalid dataset: {WORKSHOP_DATASET}")

Expected length of response rows (100) to match prefetched rows (1553). Ignoring prefetched rows.


In [16]:
if not LOAD_PRECOMPUTED_DATA:
    document_chunks = []
    for doc in data:
        # Chunk the content of each document
        chunks = chunk_simple(doc["content"], chunk_size=500, model="gpt-4o-mini")
        # Create a unique ID for the document
        doc_id = make_id(doc["content"])
        # Iterate over the chunks and create a new document for each chunk
        for i, chunk in enumerate(chunks):
            doc_chunk = deepcopy(doc)
            # Remove the content from the document
            del doc_chunk["content"]
            # Store the chunk as part of the document
            doc_chunk["chunk"] = chunk
            # Convert the chunk to text to be used as input for the LLM
            doc_chunk["text"] = convert_contents_to_text(chunk)
            # Note which chunk number it is
            doc_chunk["chunk_number"] = i
            # Create a unique ID for the chunk
            doc_chunk["chunk_id"] = make_id(chunk)
            # Add the document ID to the chunk
            doc_chunk["doc_id"] = doc_id
            # Append the chunk to the list of document chunks
            document_chunks.append(doc_chunk)

In [17]:
# Let's see the first 5 chunks
document_chunks[:5]

[{'metadata': WeaveDict({'source': '2022 Q3 AAPL.pdf', 'raw_tokens': 11081, 'file_type': 'pdf'}),
  'chunk': 'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-Q\n(Mark One)\n�� QUARTERL Y REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended June 25, 2022\nor\n�� TRANSITION REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from              to             .Commission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-24041 10\n(State or other jurisdiction\nof incorporation or organization)(I.R.S.Employer Identification No.)',
  'text': 'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-Q\n(Mark One)\n�� QUARTERL Y REPORT PURSUANT T O SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended June 25, 2022\nor\n�� TRANSITION

In [18]:
if not LOAD_PRECOMPUTED_DATA:
    dataset = weave.Dataset(name="chunked_data", rows=document_chunks)

    if SAVE_DATA:
        weave.publish(dataset)

### Building the simplest retrieval engine

One of the key ingredient of most retrieval systems is to represent the given modality (text in our case) as a vector.

This vector is a numerical representation representing the "content" of that modality (text).

Text vectorization (text to vector) can be done using various techniques like 
- [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model), 
- [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) (Term Frequency-Inverse Document Frequency), 
- and static embeddings like [Word2Vec](https://en.wikipedia.org/wiki/Word2vec), [GloVe](https://nlp.stanford.edu/projects/glove/), 
- and context aware embeddings based on transformer based models like BERT and more, which capture the semantic meaning and relationships between words or sentences.

![](./imgs/EmbeddingLayer.png)

First, we'll use TF-IDF (Term Frequency-Inverse Document Frequency) to vectorize our contents. Here's why:

- **Simplicity**: TF-IDF is straightforward to implement and understand, making it an excellent starting point for RAG systems.
- **Efficiency**: It's computationally lightweight, allowing for quick processing of large document collections.
- **No training required**: Unlike embedding models, TF-IDF doesn't need pre-training, making it easy to get started quickly.
- **Interpretability**: The resulting vectors are directly related to word frequencies, making them easy to interpret.

While more advanced methods like embeddings **might** provide better performance, especially for semantic understanding, we'll explore these in later as we progress through the workshop.


In [19]:
query = "How has Apple's total net sales changed over time?"

In [20]:
# our search engine a very simple class, that first indexes the documents and 
# then uses the search method to retrieve the top-k results 
# that are most similar to the query using cosine distance
# these are what we call sparse retrieval engines because they are based on the term frequency of the words in the documents
# they are more to do with lexical similarity than semantic similarity
tfidf_search_engine = await TfidfSearchEngine().fit(document_chunks)

# The retriever is a simple wrapper around the search engine that uses the search method
# this is mostly for the sake of consistency and to have a common interface for all the retrieval engines
class TFIDFRetriever(Retriever):
    pass

tfidf_retriever = TFIDFRetriever(search_engine=tfidf_search_engine)


retrieved_docs = await tfidf_retriever.invoke(query=query, top_k=5)
# we can look at the retrieved docs by printing them with the render_doc
# basically a fancy print statement with some formatting for the doc content and metadata
for doc in retrieved_docs:
    render_doc(doc)

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-1809-7310-8a71-db258c7a5724


- score: 0.2206
- metadata: WeaveDict({'source': '2022 Q3 AAPL.pdf', 'raw_tokens': 11081, 'file_type': 'pdf'})
- chunk_number: 23
- chunk_id: d33428f0dd6ff1183372542f1452fdb2
- doc_id: d526fb0c9708ec6e7103a267f08b9a6a


(2)Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod mini and accessories.(3)Services net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other services.Services net sales also include
amortization of the deferred value of services bundled in the sales price of certain products.iPhone
iPhone net sales increas ed during the third quarter and first nine months of 2022 compared to the same periods in 2021 due primarily to higher net sales from
the Company’ s new iPhone models.Mac
Mac net sales decrease d during the third quarter of 2022 compared to the third quarter of 2021 due primarily to lower net sales of MacBook Air and iMac ,
partially offset by higher net sales of MacBook Pro.Year-over-year Mac net sales increased durin g the first nine months of 2022 due primarily to higher net sales
of MacBook Pro, partially of fset by lower net sales of MacBook Air.iPad
iPad net sales decrease d during the third quarter of 2022 compared to the third quarter of 2021  due primarily to lower net sales of iPad Pro , partially offset by
higher net sales of all other iPad models.Year-ov er-year iPad net sales decreased during the first nine months of 2022 due primarily to lower net sales of the 10-
inch version of iPad and iPad Air, partially of fset by higher net sales of iPad mini .Wearables, Home and Accessories
Wearables, Home and Accessories net sales decreased during the third quarter of 2022 compared to the third quarter of 2021 due primarily to lower net sales of
AirPods and Apple Watch.Year-over-year Wearables, Home and Accessories net sales increased  during the first nine months of 2022 due primarily to higher net
sales of Apple W atch and AirPods.(1)
(1)
 (1)
(1)(2)
(3)
®
®
® ®
Apple Inc. | Q3 2022 Form 10-Q | 15Services
Services net sales increased during the third quarter of 2022 compared to the third quarter of 2021 due primarily to higher net sales from advertising, cloud
services and AppleCare.

---



- score: 0.1874
- metadata: WeaveDict({'source': '2023 Q1 AAPL.pdf', 'raw_tokens': 19713, 'file_type': 'pdf'})
- chunk_number: 25
- chunk_id: 08b9afbc4602c0acf9a05d7ebed88bb0
- doc_id: a1c822cc02697e5675aaa83c47357247


Rest of Asia Pacific
Rest of Asia Pacific net sales decreased during the first quarter of 2023 compared to the same quarter in 2022 due to the weakness in foreign currencies relative
to the U.S. dollar .The weakness in foreign curre ncies contributed to lower net sales of iPhone and Mac, which was partially offset by higher net sales of Services
and iPad.Products and Services Performance
The following table shows net sales by category for the three months ended December 31, 2022 and December 25, 2021 (dollars in millions):
Three Months Ended
December 31,
2022December 25,
2021 Change
Net sales by category:
iPhone $ 65,775 $ 71,628 (8)%
Mac 7,735 10,852 (29)%
iPad 9,396 7,248 30 %
Wearables, Home and Accessories 13,482 14,701 (8)%
Services 20,766 19,516 6 %
Total net sales $ 117,154 $ 123,945 (5)%
(1)Products net sales include amortization of the deferred value of unspecified software upgrade rights, which are bundled in the sales price of the respective
product.(2)Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod mini and accessories.(3)Services net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other services.Services net sales also include
amortization of the deferred value of services bundled in the sales price of certain products.iPhone
iPhone net sales decreased during the first quarter of 2023 compared to the same quarter in 2022 due primarily to lower net sales from the Company’ s new
iPhone models launched in the fourth quarter of 2022.Mac
Mac net sales decreased during the first quarter of 2023 compared to the same quarter in 2022 due primarily to lower net sales of MacBook Pro .iPad
iPad net sales increased during the first quarter of 2023 compared to the same quarter in 2022 due primarily to higher net sales of iPad and iPad Air.

---



- score: 0.1775
- metadata: WeaveDict({'source': '2023 Q3 AAPL.pdf', 'raw_tokens': 10898, 'file_type': 'pdf'})
- chunk_number: 27
- chunk_id: d4bde8367402ec76fb4ebd18b5078d6d
- doc_id: a4d28158d90e45af01c17fa1bf5fe6f9


Apple Inc. | Q3 2023 Form 10-Q | 15Products and Services Performance
The following table shows net sales by category for the three- and nine-month periods ended July 1, 2023 and June 25, 2022 (dollars in millions):
Three Months Ended Nine Months Ended
July 1,
2023June 25,
2022 ChangeJuly 1,
2023June 25,
2022 Change
Net sales by category:
iPhone $ 39,669 $ 40,665 (2)%$ 156,778 $ 162,863 (4)%
Mac 6,840 7,382 (7)% 21,743 28,669 (24)%
iPad 5,791 7,224 (20)% 21,857 22,118 (1)%
Wearables, Home and Accessories 8,284 8,084 2 % 30,523 31,591 (3)%
Services 21,213 19,604 8 % 62,886 58,941 7 %
Total net sales $ 81,797 $ 82,959 (1)%$ 293,787 $ 304,182 (3)%
iPhone
iPhone net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales from
certain iPhone models, partially of fset by higher net sales of iPhone 14 Pro models.Mac
Mac net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales of laptops.iPad
iPad net sales decreased during the third quarter of 2023 compared to the third quarter of 2022 due primarily to lower net sales across most iPad models.Year-
over-year iPad net sales were relatively flat during the first nine months of 2023.Wearables, Home and Accessories
Wearables, Home and Accessories net sales increased during the third quarter of 2023 compare d to the third quarter of 2022 due primarily to higher net sales of
Wearables, which includes AirPods , Apple Watch  and Beats  products, partially offset by lower net sales of accessories.

---



- score: 0.1719
- metadata: WeaveDict({'source': '2023 Q3 AAPL.pdf', 'raw_tokens': 10898, 'file_type': 'pdf'})
- chunk_number: 25
- chunk_id: 4bfc7956f4762e0779c6874510dbe986
- doc_id: a4d28158d90e45af01c17fa1bf5fe6f9


Segment Operating Performance
The following table shows net sales by reportable segment for the three- and nine-month periods ended July 1, 2023 and June 25, 2022 (dollars in millions):
Three Months Ended Nine Months Ended
July 1,
2023June 25,
2022 ChangeJuly 1,
2023June 25,
2022 Change
Net sales by reportable segment:
Americas $ 35,383 $ 37,472 (6)%$ 122,445 $ 129,850 (6)%
Europe 20,205 19,287 5 % 71,831 72,323 (1)%
Greater China 15,758 14,604 8 % 57,475 58,730 (2)%
Japan 4,821 5,446 (11)% 18,752 20,277 (8)%
Rest of Asia Pacific 5,630 6,150 (8)% 23,284 23,002 1 %
Total net sales $ 81,797 $ 82,959 (1)%$ 293,787 $ 304,182 (3)%
Americas
Americas net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales of
iPhone and Mac, partially of fset by higher net sales of Services.Europe
The weakness in foreign  currencies relative to the U.S. dollar had a net unfavorable year-over-y ear impact on Europe net sales during the third quarter and first
nine months  of 2023.During the third quarter of 2023, the Europe net sales increase consiste d primarily of higher net sales of iPhone.During the first nine
months of 2023, the Europe net sales decrease consisted primarily of lower net sales of Mac, partially of fset by higher net sales of iPhone.Greater China
The weakness in the renminbi relative to the U.S. dollar had an unfavorable year-over-year impact on Greater China net sales during the third quarter and first
nine months of 2023.During the third quarter of 2023, the Greater China net sales increase consisted primarily of higher net sales of iPhone.During the first
nine months of 2023, the Greater China net sales decrease consisted primarily of lower net sales of iPhone.

---



- score: 0.1602
- metadata: WeaveDict({'source': '2022 Q3 AAPL.pdf', 'raw_tokens': 11081, 'file_type': 'pdf'})
- chunk_number: 25
- chunk_id: fed3877be83a64ed1694fddbcf141f7c
- doc_id: d526fb0c9708ec6e7103a267f08b9a6a


Furth er informat ion regarding the Com pany’ s reportable segments can be found in Part I, Item 1 of this Form 10-Q in the Notes to Condensed
Consolidated Financial Statements in Note 9, “Segment Information and Geographic Data.”
The following table shows net sales by reportable segment for the three- and nine-month periods ended June 25, 2022 and June 26, 2021 (dollars in millions):
Three Months Ended Nine Months Ended
June 25,
2022June 26,
2021 ChangeJune 25,
2022June 26,
2021 Change
Net sales by reportable segment:
Americas $ 37,472 $ 35,870 4 %$ 129,850 $ 116,486 11 %
Europe 19,287 18,943 2 % 72,323 68,513 6 %
Greater China 14,604 14,762 (1)% 58,730 53,803 9 %
Japan 5,446 6,464 (16)% 20,277 22,491 (10)%
Rest of Asia Pacific 6,150 5,395 14 % 23,002 21,164 9 %
Total net sales $ 82,959 $ 81,434 2 %$ 304,182 $ 282,457 8 %
Americas
Americas net sales increased during the third quarter of 2022 compared to the third quarter of 2021 due primarily to higher net sales of Services and iPhone,
partially offset by lower net sales of Mac and Wearables, Home and Accessories.Year-over-ye ar Americas net sales increased during the first nine months of
2022 due primarily to higher net sales of iPhone, Services and W earables, Home and Accessories.Europe
Europe net sales increas ed during the third quarter of 2022 compared to the third quarter of 2021 due primarily to higher net sales of iPhone, partially offset by
lower net sales of Mac.Year-over-year Europe net sales increased during the first nine months of 2022 due primarily to higher net sales of Services and iPhone.The weakness in foreign  currencies relative to the U.S. dollar had a net unfavorable impact on Europe net sales during the third quarter and first nine months of
2022.

---



Note that the `Retriever` class above is inherited from `weave.Model`.

A `Model` is a combination of data (which can include configuration, trained model weights, or other information) and code that defines how the model operates. By structuring your code to be compatible with this API, you benefit from a structured way to version your application so you can more systematically keep track of your experiments.

To create a model in Weave, you need the following:

- a class that inherits from `weave.Model`
- type definitions on all attributes
- a typed `predict`, `invoke` or `forward` method with `@weave.op` decorator.

Imagine `weave.op` to be a drop in replacement for `print` or logging statement.
However, it does a lot more than just printing and logging by tracking the inputs and outputs of the function and storing them as Weave objects.
In addition to state tracking you also get a nice weave UI to inspect the inputs, outputs, and other metadata.

If you have not initialized a weave run by doing `weave.init`, the code will work as it is without any tracking.

The `predict` method decorated with `weave.op()` will track the model settings along with the inputs and outputs anytime you call it.

### Generating a response

There are two components of any RAG pipeline - a `Retriever` and a `ResponseGenerator`. Having designed a simple retriever, here we are designing a `SimpleResponseGenerator`.

The `SimpleResponseGenerator` takes the user question along with the retrieved context (documents) as inputs and makes a LLM call using the `model` and `prompt` (system prompt). This way the generated answer is grounded on the documentation (our usecase). In this case we are using Cohere's `command-r` model.

As earlier, we have wrapped this `SimpleResponseGenerator` class with weave for tracking the inputs and the output.

The `ResponseGenerator` also has a `system_prompt` that we pass to the LLM. Consider this to be set of instructions we give to the LLM on what to do with the user's query and the retrieved documents.
In practice, the system prompt can be detailed and involved (depending on the usecase) but we are using a very simple prompt here.
Later we will iterate on it and show how improving the system prompt improves the quality of the generated response.

In [21]:
docs_data = [{"document": item["text"]} for item in retrieved_docs]
simple_response_generator = SimpleResponseGenerator()
response = await simple_response_generator.invoke(
    query=query, documents=docs_data
)
printmd(response["choices"][0]["message"]["content"])

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-186a-76d3-b56c-6088f9d5926f


Apple's total net sales have shown fluctuations over time, as indicated by the provided financial data. For the three-month period ended December 31, 2022, total net sales were $117,154 million, which was a decrease from $123,945 million for the same period in 2021, reflecting a 5% decline. 

In the following quarters, for the three months ended July 1, 2023, total net sales were reported at $81,797 million, slightly down from $82,959 million in the same period of 2022, which indicates a 1% decrease. Over the nine-month periods, total net sales decreased from $304,182 million in 2022 to $293,787 million in 2023, a decline of 3%.

Overall, the trend shows a decrease in total net sales in certain quarters compared to previous years, indicating variability in sales performance across different product categories and time periods.

### Bringing everything together

Finally, with all the components ready, we can bring everything together into a single pipeline.

Here, we define a `weave.Model` class `SimpleRAGPipeline` which combines the steps of retrieval and response generation.

We'll define a `invoke` method that takes the user query, retrieves relevant context using the retriever and finally synthesizes a response using the response generator.

We'll also define a few convinence methods to format the documents retrieved from the retriever and create a system prompt for the response generator.

In [22]:
# we are using the SimpleRAGPipeline class to combine the steps of retrieval and response generation

class TFIDFRAGPipeline(SimpleRAGPipeline):
    pass

# Instantiate the pipeline with the retriever and response generator
tfidf_rag_pipeline = TFIDFRAGPipeline(
    retriever=tfidf_retriever,
    generator=simple_response_generator
)

# Invoke the pipeline with the user query
response = await tfidf_rag_pipeline.invoke(query=query,)
printmd(response["answer"])

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-2957-7f81-8d2d-6fc60f5a44c7


Based on the provided documents, Apple's total net sales have experienced fluctuations over time. Here are some key points regarding the changes in total net sales:

1. **Q3 2022**: Total net sales were $82,959 million, which was an increase from the previous year (Q3 2021) when they were $81,434 million, representing a year-over-year increase of approximately 2%.

2. **Q1 2023**: Total net sales decreased to $117,154 million compared to $123,945 million in Q1 2022, showing a decline of about 5%.

3. **Q3 2023**: Total net sales slightly decreased to $81,797 million compared to $82,959 million in Q3 2022, reflecting a 1% decrease. However, for the nine-month period ending July 1, 2023, total net sales were $293,787 million, which is down from $304,182 million for the same period in the previous year, indicating a 3% decline.

Overall, while there were periods of growth in Apple's total net sales, there have also been declines in certain quarters, particularly when comparing year-over-year figures.

## Evaluating the RAG Pipeline

Before we starting making further changes to the RAG pipeline, it is a good practice to evaluate the current state of the pipeline
This will help us understand the current performance of the pipeline and identify areas of improvement.
Think of evaluation as a way to measure the performance of your system - a benchmark to compare the performance of your system against.

Evaluating a RAG pipeline is a crucial step in ensuring its robustness and effectiveness.
We will evaluate the two main components of a RAG pipeline - retriever and response generator.

Evaluating the retriever can be considered component evaluation. 
Depending on your RAG pipeline, there can be a few components and for ensuring robustness of your system,
it is recommended to come up with evaluation for each component.

![](./imgs/EvolvingRAG.png)


### Collecting data for evaluation

We are using a subset of the evaluation dataset we had created for wandbot.

Learn more about how we created the evaluation dataset here:

- [How to Evaluate an LLM, Part 1: Building an Evaluation Dataset for our LLM System](https://wandb.ai/wandbot/wandbot-eval/reports/How-to-Evaluate-an-LLM-Part-1-Building-an-Evaluation-Dataset-for-our-LLM-System--Vmlldzo1NTAwNTcy)
- [How to Evaluate an LLM, Part 2: Manual Evaluation of Wandbot, our LLM-Powered Docs Assistant](https://wandb.ai/wandbot/wandbot-eval/reports/How-to-Evaluate-an-LLM-Part-2-Manual-Evaluation-of-Wandbot-our-LLM-Powered-Docs-Assistant--Vmlldzo1NzU4NTM3)

The main take away from these reports are:

- we first deployed wandbot for internal usage based on rigorous eyeballing based evalution.
- the user query distribution was throughly analyzed and clustered. we sampled a good representative queries from these clusters and created a gold standard set of queries.
- we then used in-house MLEs to perform manual evaluation using Argilla. Creating such evaluation platforms are easy.
- To summarize, speed is the key here. Use whatever means you have to create a meaningful eval set.

The evaluation samples are logged as [`weave.Dataset`](https://wandb.github.io/weave/guides/core-types/datasets/). `weave.Dataset` enables you to collect examples for evaluation and automatically track versions for accurate comparisons.

Below we will download the latest version locally with a simple API.

In [23]:
# here we are downloading the latest version of the dataset from Weave
if WORKSHOP_DATASET == "financebench":
    eval_dataset = weave.ref("weave:///a-sh0ts/rag-workshop-pc-nyc-financebench/object/financebench_eval_dataset:9lYVoXzGjqcmd4tiCd4SvwrW3T8pdo9J4wYbgBKDwUc").get()
elif WORKSHOP_DATASET == "sec-10-q":
    eval_dataset = weave.ref("weave:///a-sh0ts/rag-course-finance/object/eval_data:CoQDvdOENbZqkwg7IlhZm33drBCAf9OUNvf8ar6YHzM").get()

print("Number of evaluation samples: ", len(eval_dataset.rows))

Number of evaluation samples:  9


Iterating through each sample is easy.

We have the question, ground truth answer and ground truth contexts.

In [24]:
eval_dataset.rows[0]

WeaveDict({'question': "How has Apple's total net sales changed over time?", 'answer': "Based on the provided documents, Apple's total net sales have changed over time as follows:\n\n- For the quarterly period ended June 25, 2022, the total net sales were $82,959 million. (SOURCE: 2022 Q3 AAPL.pdf)\n- For the quarterly period ended December 31, 2022, the total net sales were $117,154 million. (SOURCE: 2023 Q1 AAPL.pdf)\n- For the quarterly period ended April 1, 2023, the total net sales were $94,836 million. (SOURCE: 2023 Q2 AAPL.pdf)\n- For the quarterly period ended July 1, 2023, the total net sales were $81,797 million. (SOURCE: 2023 Q3 AAPL.pdf)\n\nFrom these figures, it can be observed that there was an increase in total net sales from the quarter ended June 25, 2022, to the quarter ended December 31, 2022. However, there was a subsequent decrease in total net sales in the quarters ended April 1, 2023, and July 1, 2023.\n\nSOURCE(S): 2022 Q3 AAPL.pdf, 2023 Q1 AAPL.pdf, 2023 Q2 AAP

Again, we'll use W&B Weave for our evaluation purposes.
The `weave.Evaluation` class is a light weight class that can be used to evaluate the performance of a `weave.Model` on a `weave.Dataset`.

### Evaluating the Retriever

The fundamental idea of evaluating a retriever is to check how well the retrieved content matches the expected contents.
To evaluate a RAG pipeline, we need query and ground truth answer pairs. 
The ground truth answer must be grounded on some "ground" truth chunks.
This is a search problem, it's easiest to start with traditional Information retrieval metrics.



You might already have access to such evaluation dataset depending on the nature of your application or you can synthetically build one. 
To build one you can retrieve random documents/chunks and ask an LLM to generate query-answer pairs - the underlying documents/chunks will act as your ground truth chunk.

Here, we will look at different metrics that can be used to evaluate the retriever we built earlier.

### Metrics to evaluate retriever

We can evaluate a retriever using traditional ML metrics. We can also evaluate it by using a powerful LLM (next section).

Below we are importing both traditional metrics and classifier based metric from the [`retrieval_metrics.py`](retrieval_metrics.py) file.

* **Hit Rate**: Measures the proportion of queries where the retriever successfully returns at least one relevant document.
* **MRR (Mean Reciprocal Rank)**: Evaluates how quickly the retriever returns the first relevant document, based on the reciprocal of its rank.
* **NDCG (Normalized Discounted Cumulative Gain)**: Assesses the quality of the ranked retrieval results, giving more importance to relevant documents appearing earlier.
* **MAP (Mean Average Precision)**: Computes the mean precision across all relevant documents retrieved, considering the rank of each relevant document.
* **Precision**: Measures the ratio of relevant documents retrieved to the total documents retrieved by the retriever.
* **Recall**: Evaluates the ratio of relevant documents retrieved to the total relevant documents available for the query.
* **F1 Score**: The harmonic mean of precision and recall, providing a balance between both metrics to gauge retriever performance.
* **Relevance Score**: A binary score predicted by a pre-trained classifier that measures the relevance of the retrieved documents to the query.

We will be using the `RetrievalScorer` class to evaluate the retriever.
Each metric expects an `output` which is a list of retrieved chunks from the retriever and `contexts` which is a list of ground truth contexts from the evaluation dataset.


In [25]:
retrieval_evaluation = weave.Evaluation(
    name="Retrieval_Evaluation",
    dataset=eval_dataset,
    scorers=[RetrievalScorer(name="retrieval_scorer", description="Retrieval metrics")],
    preprocess_model_input=lambda x: {"query": x["question"], "top_k": 5},
)

In [26]:
tfidf_retrieval_scores = await retrieval_evaluation.evaluate(
    model=tfidf_retriever,
    __weave={"display_name": "TFIDF Retrieval"}
)


Compiling the model with `torch.compile` and using a `torch.mps` device is not supported. Falling back to non-compiled mode.


Evaluated 1 of 9 examples

Evaluated 2 of 9 examples

Evaluated 3 of 9 examples

Evaluated 4 of 9 examples

...

Evaluated 7 of 9 examples

Evaluated 8 of 9 examples

Evaluated 9 of 9 examples

Evaluation summary
{
    'retrieval_scorer': {
        'precision': {'mean': 0.4740740740740741},
        'hit_rate': {'mean': 0.4666666666666667},
        'map': {'mean': 0.5819444444444444},
        'relevance': {'mean': 0.0},
        'relevance_score': {'mean': 0.25402222222222215},
        'mrr': {'mean': 0.21222222222222223},
        'ndcg': {'mean': 0.4775712605563381},
        'f1_score': {'mean': 0.4645502645502645},
        'recall': {'mean': 0.4722222222222222}
    },
    'model_latency': {'mean': 0.09644577238294813}
}

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-3e3f-7270-9cf7-f7f021a47174


### Evaluating the Response Generation

Evaluating the generated response is a bit more complex. Although we have some ground truth answers, they are not directly comparable to the generated response. This could be due to a few reasons:

- The generated response might be a paraphrased version of the ground truth answer.
- The generated response might be a more detailed version of the ground truth answer.
- The generated response might be a more concise version of the ground truth answer.
- The generated response might be a more structured version of the ground truth answer.


Some of these metrics are based on the distance between the generated response and the ground truth answer.
- **Diff Score**: compute the similarity ratio between the normalized model output and the expected answer.
- **Levenshtein Score**: compute the Levenshtein ratio between the normalized model output and the answer.
- **ROUGE Score**: compute the ROUGE-L F1 score between the normalized model output and the reference answer.
- **BLEU Score**: compute the BLEU score between the normalized model output and the reference answer.
- **METEOR Score**: compute the METEOR score between the normalized model output and the reference answer.
- **Correctness Score**: a binary score predicted by a pre-trained classifier that measures the correctness of the generated response.
- **Helpfulness Score**: a binary score predicted by a pre-trained classifier that measures the helpfulness of the generated response.
- **Relevance Score**: a binary score predicted by a pre-trained classifier that measures the relevance of the generated response to the query.

While these metrics are not perfect, they provide a good north star for evaluating the response generation and can be used to compare the performance of the response generation model.

We will be using the `ResponseScorer` class to evaluate the response generation.
Each metric expects an `output` containing the generated response and the ground truth `answer`.

In [27]:
# Again, we are using the `weave.Evaluation` class to evaluate the response generation.
response_evaluation = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=[ResponseScorer(name="response_scorer", description="Response metrics")],
    preprocess_model_input=lambda x: {"query": x["question"]},
)


In [28]:
tfidf_response_scores = await response_evaluation.evaluate(
    model=tfidf_rag_pipeline,
    __weave={"display_name": "TFIDF RAG Pipeline"}
)

Evaluated 1 of 9 examples

Evaluated 2 of 9 examples

Evaluated 3 of 9 examples

Evaluated 4 of 9 examples

Evaluated 5 of 9 examples

...

Evaluated 7 of 9 examples

Evaluated 8 of 9 examples

Evaluated 9 of 9 examples

Evaluation summary
{
    'response_scorer': {
        'levenshtein': {'mean': 0.4786544443961286},
        'rouge': {'mean': 0.27619175049180633},
        'correctness': {
            'correct': {'true_count': 4, 'true_fraction': 0.4444444444444444},
            'extras': {'label': {'mean': 0.4444444444444444}, 'score': {'mean': 0.5839604271782769}}
        },
        'helpfulness': {
            'helpful': {'true_count': 6, 'true_fraction': 0.6666666666666666},
            'extras': {'label': {'mean': 0.6666666666666666}, 'score': {'mean': 0.7262296080589294}}
        },
        'bleu': {'mean': 0.1082992182370372},
        'relevance': {
            'extras': {'label': {'mean': 0.6666666666666666}, 'score': {'mean': 0.7022299104266696}},
            'relevant': {'true_count': 6, 'true_fraction': 0.6666666666666666}
        },
        'diff': {'mean': 0.08479422258454458},
        'meteor': {'mean': 0.29596666666666666}
    },
    'model_latency': {'mean': 3.429615709516737}
}

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-62ec-7fc1-a221-091d295a1ecf


## Improving the RAG Pipeline

One of the most common ways to improve the performance of a RAG pipeline is to use a better retrieval algorithm. It is the low hanging fruit and often the first thing to try. This is because the retrieval step is one of the most important steps in a RAG pipeline. With a good retrieval algorithm, we can retrieve more relevant documents and improve the quality and relevance of the generated response.

### Using BM25 for Retrieval

BM25 is a popular retrieval algorithm that uses the term frequency of the words in the documents to retrieve the most relevant documents. 
Many production IR systems use BM25.

For instance Elasticsearch uses BM25 as its default retrieval algorithm. 
It's a simple algorithm that is easy to implement and has been shown to be effective in many cases.
Although it doesn't perform semantic search, it's a good starting point for improving the retrieval step.
You'll be suprised how much it alone can improve the performance of the RAG pipeline.

We will be using the `BM25SearchEngine` class to create a BM25 search engine.


In [29]:
# # we are still fitting the search engine on the same document chunks as we did for the TFIDF search engine
bm25_search_engine = await BM25SearchEngine().fit(document_chunks)

# A wrapper around the search engine that uses the search method
class BM25Retriever(Retriever):
    pass


bm25_retriever = BM25Retriever(search_engine=bm25_search_engine)

retrieved_docs = await bm25_retriever.invoke(query=query, top_k=5)

for doc in retrieved_docs:
    render_doc(doc)


🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-84f7-7bb0-94de-776edc620245


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


- score: 5.6303
- metadata: WeaveDict({'source': '2023 Q2 AAPL.pdf', 'raw_tokens': 10826, 'file_type': 'pdf'})
- chunk_number: 23
- chunk_id: f0fc181ba63b9d137a32b7f1bdc9a8b5
- doc_id: 892ac7f7702cd56f574dae211e89bfa9


Available Information
The Company periodically provides certain information for investors on its corporate website, www .apple.com, and its investor relations website,
investor .apple.com.This includes press releases and other information about financial performance, information on environmental, social and governance
matters, and details related to the Company’ s annual meeting of shareholders.The information contained on the websites referenced in this Form 10-Q is not
incorporated by reference into this filing.Further , the Company’ s references to website URLs are intended to be inactive textual references only .Business Seasonality and Product Introductions
The Company has historically experienced higher net sales in its first quarter compared to other quarters in its fiscal year due in part to seasonal holiday
demand.Additionally , new product and service introductions can significantly impact net sales , cost of sales and operating expenses.The timing of product
introductions can also impact the Company’ s net sales to its indirect distribution channels as these channels are filled with new inventory following a product
launch, and channel inventory of an older product often declines as the launch of a newer produ ct approaches.Net sales can also be affected when consumers
and distributors anticipate a product introduction.Fiscal Period
The Compan y’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September .An additional week is included in the first fiscal quarter
every five or six years to realign the Company’ s fiscal quarters with calendar quarters, which occurred in the first quarter of 2023.The Company’ s fiscal years
2023 and 2022 span 53 and 52 weeks, respectively .Quarterly Highlights
Weakness in foreign currencies relative to the U.S. dollar had an unfavorable impact on the Com pany’ s total net sales, which decreased 3% or $2.4 billion during
the second quarter of 2023 compared to the same quarter in 2022.The year-over-year net sales decrease consisted primarily of lower net sales of Mac, partially
offset by higher net sales of Services.During the second quarter of 2023, the Company announced the following new products:
•MacBook Pro  14” and MacBook Pro 16”, powered by the Apple M2 Pro and M2 Max chip;
•Mac mini , powered by the Apple M2 and M2 Pro chip; and
•Second-generation HomePod .

---



- score: 5.3702
- metadata: WeaveDict({'source': '2023 Q3 AAPL.pdf', 'raw_tokens': 10898, 'file_type': 'pdf'})
- chunk_number: 23
- chunk_id: ee0556acf374c90418fef0e1d28d98b2
- doc_id: a4d28158d90e45af01c17fa1bf5fe6f9


Available Information
The Company periodically provides certain information for investors on its corporate website, www .apple.com, and its investor relations website,
investor .apple.com.This includes press releases and other information about financial performance, information on environmental, social and governance
matters, and details related to the Company’ s annual meeting of shareholders.The information contained on the websites referenced in this Form 10-Q is not
incorporated by reference into this filing.Further , the Company’ s references to website URLs are intended to be inactive textual references only .Business Seasonality and Product Introductions
The Company has historically experienced higher net sales in its first quarter compared to other quarters in its fiscal year due in part to seasonal holiday
demand.Additionally , new product and service introductions can significantly impact net sales , cost of sales and operating expenses.The timing of product
introductions can also impact the Company’ s net sales to its indirect distribution channels as these channels are filled with new inventory following a product
launch, and channel inventory of an older product often declines as the launch of a newer produ ct approaches.Net sales can also be affected when consumers
and distributors anticipate a product introduction.Fiscal Period
The Compan y’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September .An additional week is included in the first fiscal quarter
every five or six years to realign the Company’ s fiscal quarters with calendar quarters, which occurred in the first quarter of 2023.The Company’ s fiscal years
2023 and 2022 span 53 and 52 weeks, respectively .Quarterly Highlights
Weakness in foreign currencies relative to the U.S. dollar had an unfavorable impact on the Com pany’ s total net sales, which decreased 1% or $1.2 billion during
the third quarter of 2023 compared to the same quarter in 2022.The year-over-year net sales decrease consisted primarily of lower net sales of iPad and
iPhone, partially of fset by higher net sales of Services.

---



- score: 5.185
- metadata: WeaveDict({'source': '2022 Q3 AAPL.pdf', 'raw_tokens': 11081, 'file_type': 'pdf'})
- chunk_number: 23
- chunk_id: d33428f0dd6ff1183372542f1452fdb2
- doc_id: d526fb0c9708ec6e7103a267f08b9a6a


(2)Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod mini and accessories.(3)Services net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other services.Services net sales also include
amortization of the deferred value of services bundled in the sales price of certain products.iPhone
iPhone net sales increas ed during the third quarter and first nine months of 2022 compared to the same periods in 2021 due primarily to higher net sales from
the Company’ s new iPhone models.Mac
Mac net sales decrease d during the third quarter of 2022 compared to the third quarter of 2021 due primarily to lower net sales of MacBook Air and iMac ,
partially offset by higher net sales of MacBook Pro.Year-over-year Mac net sales increased durin g the first nine months of 2022 due primarily to higher net sales
of MacBook Pro, partially of fset by lower net sales of MacBook Air.iPad
iPad net sales decrease d during the third quarter of 2022 compared to the third quarter of 2021  due primarily to lower net sales of iPad Pro , partially offset by
higher net sales of all other iPad models.Year-ov er-year iPad net sales decreased during the first nine months of 2022 due primarily to lower net sales of the 10-
inch version of iPad and iPad Air, partially of fset by higher net sales of iPad mini .Wearables, Home and Accessories
Wearables, Home and Accessories net sales decreased during the third quarter of 2022 compared to the third quarter of 2021 due primarily to lower net sales of
AirPods and Apple Watch.Year-over-year Wearables, Home and Accessories net sales increased  during the first nine months of 2022 due primarily to higher net
sales of Apple W atch and AirPods.(1)
(1)
 (1)
(1)(2)
(3)
®
®
® ®
Apple Inc. | Q3 2022 Form 10-Q | 15Services
Services net sales increased during the third quarter of 2022 compared to the third quarter of 2021 due primarily to higher net sales from advertising, cloud
services and AppleCare.

---



- score: 5.006
- metadata: WeaveDict({'source': '2023 Q3 AAPL.pdf', 'raw_tokens': 10898, 'file_type': 'pdf'})
- chunk_number: 27
- chunk_id: d4bde8367402ec76fb4ebd18b5078d6d
- doc_id: a4d28158d90e45af01c17fa1bf5fe6f9


Apple Inc. | Q3 2023 Form 10-Q | 15Products and Services Performance
The following table shows net sales by category for the three- and nine-month periods ended July 1, 2023 and June 25, 2022 (dollars in millions):
Three Months Ended Nine Months Ended
July 1,
2023June 25,
2022 ChangeJuly 1,
2023June 25,
2022 Change
Net sales by category:
iPhone $ 39,669 $ 40,665 (2)%$ 156,778 $ 162,863 (4)%
Mac 6,840 7,382 (7)% 21,743 28,669 (24)%
iPad 5,791 7,224 (20)% 21,857 22,118 (1)%
Wearables, Home and Accessories 8,284 8,084 2 % 30,523 31,591 (3)%
Services 21,213 19,604 8 % 62,886 58,941 7 %
Total net sales $ 81,797 $ 82,959 (1)%$ 293,787 $ 304,182 (3)%
iPhone
iPhone net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales from
certain iPhone models, partially of fset by higher net sales of iPhone 14 Pro models.Mac
Mac net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales of laptops.iPad
iPad net sales decreased during the third quarter of 2023 compared to the third quarter of 2022 due primarily to lower net sales across most iPad models.Year-
over-year iPad net sales were relatively flat during the first nine months of 2023.Wearables, Home and Accessories
Wearables, Home and Accessories net sales increased during the third quarter of 2023 compare d to the third quarter of 2022 due primarily to higher net sales of
Wearables, which includes AirPods , Apple Watch  and Beats  products, partially offset by lower net sales of accessories.

---



- score: 4.8996
- metadata: WeaveDict({'source': '2022 Q3 AAPL.pdf', 'raw_tokens': 11081, 'file_type': 'pdf'})
- chunk_number: 21
- chunk_id: b2d7313fb7914541a5f775519a2a3fe5
- doc_id: d526fb0c9708ec6e7103a267f08b9a6a


Available Information
The Company periodically provides certain information for investors on its corporate website, www .apple.com, and its investor relations website,
investor .apple.com.This includes press releases and other information about financial performance, information on environmental, social and corporate
governance matters, and details related to the Company’ s annual meeting of shareholders.The information contained on the websites referenced in this Form
10-Q is not incorporated by reference into this filing.Further , the Company’ s references to website URLs are intended to be inactive textual references only .Business Seasonality and Product Introductions
The Company has historically experienced higher net sales in its first quarter compared to other quarters in its fiscal year due in part to seasonal holiday
demand.Additionally , new product and service introductions can significantly impact net sales , cost of sales and operating expenses.The timing of product
introductions can also impact the Company’ s net sales to its indirect distribution channels as these channels are filled with new inventory following a product
launch, and channel inventory of an older product often declines as the launch of a newer produ ct approaches.Net sales can also be affected when consumers
and distributors anticipate a product introduction.Quarterly Highlights
Total net sales increased 2% or $1.5 billion during the third quarter of 2022 compared to the same quarter in 2021, driven primarily by higher net sales of
Services and iPhone, partially offset by lower net sales of Mac and Wearables, Home and Accessories.The weakness in foreign currencies relative to the U.S.
dollar had an unfavorable impact on all Products and Services net sales during the third quarter of 2022.At the end of the third quarter of 2022, the Com pany introduced an all-new MacBook Air and an updated 13-inch  MacBook Pro , both powe red by the new
Apple M2 chip.The Company also announced iOS 16, macOS  Ventura, iPadOS  16 and watchOS  9, updates to its operating systems that are expe cted to be
available in the fall of 2022.

---



In [30]:
# we are now ready to evaluate the retrieval performance of the BM25 retrieval algorithm
# we are using the same evaluation dataset as we did for the TFIDF retrieval algorithm
# this should allow us to compare the performance of the two retrieval algorithms
retrieval_scores = await retrieval_evaluation.evaluate(
    model=bm25_retriever,
    __weave={"display_name": "BM25 Retrieval"}
)

Evaluated 1 of 9 examples

Evaluated 2 of 9 examples

Evaluated 3 of 9 examples

Evaluated 4 of 9 examples

Evaluated 5 of 9 examples

...

Evaluated 7 of 9 examples

Evaluated 8 of 9 examples

Evaluated 9 of 9 examples

Evaluation summary
{
    'retrieval_scorer': {
        'precision': {'mean': 0.4666666666666667},
        'hit_rate': {'mean': 0.4666666666666667},
        'map': {'mean': 0.5694444444444444},
        'relevance': {'mean': 0.0},
        'relevance_score': {'mean': 0.2924333333333333},
        'mrr': {'mean': 0.2085185185185185},
        'ndcg': {'mean': 0.465356140554866},
        'f1_score': {'mean': 0.4444444444444444},
        'recall': {'mean': 0.4444444444444444}
    },
    'model_latency': {'mean': 0.0025416745079888236}
}

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-850a-72d0-b13f-5d7f3c7e1582


While evaluating the retrieval performance is important, we are interested more in the overall performance of the RAG pipeline.

Again, we will be using the `SimpleRAGPipeline` class to create a RAG pipeline using the same response generator as we did earlier.

We are also using the same evaluation dataset as we did for the TFIDF RAG pipeline. This should allow us to compare the performance of the two pipelines.

In [31]:
class BM25RAGPipeline(SimpleRAGPipeline):
    pass


bm25_rag_pipeline = BM25RAGPipeline(
    retriever=bm25_retriever,
    generator=simple_response_generator
)

response = await bm25_rag_pipeline.invoke(query=query)

printmd(response["answer"])

response_scores = await response_evaluation.evaluate(
    model=bm25_rag_pipeline,
    __weave={"display_name": "BM25 RAG Pipeline"}
)

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-aae1-7e20-b913-3973da014534


Based on the provided documents, Apple's total net sales have experienced fluctuations over different quarters and years. Here are some key highlights:

1. **Q2 2023**: Apple's total net sales decreased by 3% or $2.4 billion compared to the same quarter in 2022. This decrease was primarily due to lower net sales of Mac, although it was partially offset by higher net sales of Services.

2. **Q3 2023**: Total net sales decreased by 1% or $1.2 billion compared to the same quarter in 2022. The decline was driven by lower net sales of iPad and iPhone, again offset by increases in Services.

3. **Q3 2022**: Compared to Q3 2021, total net sales increased by 2% or $1.5 billion, primarily driven by higher sales of Services and iPhone, despite lower sales of Mac and Wearables.

4. **Q1 2023**: The fiscal year includes seasonal trends, with historically higher net sales in the first quarter due to holiday demand, but specific figures for this quarter are not provided in the documents.

Overall, while there have been periods of growth in total net sales, recent quarters have shown a trend of declining sales compared to previous years, particularly influenced by product-specific performance and external factors like currency fluctuations.

Evaluated 1 of 9 examples

Evaluated 2 of 9 examples

Evaluated 3 of 9 examples

...

Evaluated 7 of 9 examples

Evaluated 8 of 9 examples

Evaluated 9 of 9 examples

Evaluation summary
{
    'response_scorer': {
        'levenshtein': {'mean': 0.46122580299789045},
        'rouge': {'mean': 0.24124048407882295},
        'correctness': {
            'correct': {'true_count': 3, 'true_fraction': 0.3333333333333333},
            'extras': {'label': {'mean': 0.3333333333333333}, 'score': {'mean': 0.5612127516004775}}
        },
        'helpfulness': {
            'helpful': {'true_count': 5, 'true_fraction': 0.5555555555555556},
            'extras': {'label': {'mean': 0.5555555555555556}, 'score': {'mean': 0.6966763999727037}}
        },
        'bleu': {'mean': 0.10466258209035638},
        'relevance': {
            'extras': {'label': {'mean': 0.5555555555555556}, 'score': {'mean': 0.6284174654218886}},
            'relevant': {'true_count': 5, 'true_fraction': 0.5555555555555556}
        },
        'diff': {'mean': 0.0672449432749076},
        'meteor': {'mean': 0.27196666666666663}
    },
    'model_latency': {'mean': 3.828370147281223}
}

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbc-c04c-7f50-a96b-524df962fd07


### Using Dense Retrieval

Dense retrieval is a retrieval algorithm that uses the embeddings of the documents to retrieve the most relevant documents.

While technically we can use any machine learning model to create embeddings, we will use the embeddings created by some pre-trained embedding model here. Specifically, we will use the embeddings created by the `MSFT -> multilingual-e5-large` embed API on Pinecone.

We model this using the `DenseSearchEngine` class to create a search engine that uses the embeddings to retrieve the most relevant documents using cosine distance.

The retrieved documents are more likely to be semantically relevant than being lexically similar i.e. they are likely to be similar to the query in terms of the semantic meaning of the words rather than the exact words that are used in the query.


In [32]:
# we are fitting the search engine on the same document chunks as we did earlier
dense_search_engine = DenseSearchEngine()
dense_search_engine = await dense_search_engine.fit(document_chunks)

# Another wrapper around the search engine
class DenseRetriever(Retriever):
    pass

dense_retriever = DenseRetriever(search_engine=dense_search_engine)

retrieved_docs = await dense_retriever.invoke(query=query, top_k=5)
for doc in retrieved_docs:
    render_doc(doc)


# Let's evaluate the retrieval performance of the dense retrieval algorithm
retrieval_scores = await retrieval_evaluation.evaluate(
    model=dense_retriever,
    __weave={"display_name": "Dense Retrieval"}
)


# The same drill, we create a RAG pipeline and evaluate it.
class DenseRAGPipeline(SimpleRAGPipeline):
    pass

dense_rag_pipeline = DenseRAGPipeline(
    retriever=dense_retriever,
    generator=simple_response_generator
)

response_scores = await response_evaluation.evaluate(
    model=dense_rag_pipeline,
    __weave={"display_name": "Dense RAG Pipeline"}
)


🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbf-78bc-72b3-aa68-f2ae8584d3cb


- score: 0.8334
- metadata: WeaveDict({'source': '2022 Q3 AAPL.pdf', 'raw_tokens': 11081, 'file_type': 'pdf'})
- chunk_number: 23
- chunk_id: d33428f0dd6ff1183372542f1452fdb2
- doc_id: d526fb0c9708ec6e7103a267f08b9a6a


(2)Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod mini and accessories.(3)Services net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other services.Services net sales also include
amortization of the deferred value of services bundled in the sales price of certain products.iPhone
iPhone net sales increas ed during the third quarter and first nine months of 2022 compared to the same periods in 2021 due primarily to higher net sales from
the Company’ s new iPhone models.Mac
Mac net sales decrease d during the third quarter of 2022 compared to the third quarter of 2021 due primarily to lower net sales of MacBook Air and iMac ,
partially offset by higher net sales of MacBook Pro.Year-over-year Mac net sales increased durin g the first nine months of 2022 due primarily to higher net sales
of MacBook Pro, partially of fset by lower net sales of MacBook Air.iPad
iPad net sales decrease d during the third quarter of 2022 compared to the third quarter of 2021  due primarily to lower net sales of iPad Pro , partially offset by
higher net sales of all other iPad models.Year-ov er-year iPad net sales decreased during the first nine months of 2022 due primarily to lower net sales of the 10-
inch version of iPad and iPad Air, partially of fset by higher net sales of iPad mini .Wearables, Home and Accessories
Wearables, Home and Accessories net sales decreased during the third quarter of 2022 compared to the third quarter of 2021 due primarily to lower net sales of
AirPods and Apple Watch.Year-over-year Wearables, Home and Accessories net sales increased  during the first nine months of 2022 due primarily to higher net
sales of Apple W atch and AirPods.(1)
(1)
 (1)
(1)(2)
(3)
®
®
® ®
Apple Inc. | Q3 2022 Form 10-Q | 15Services
Services net sales increased during the third quarter of 2022 compared to the third quarter of 2021 due primarily to higher net sales from advertising, cloud
services and AppleCare.

---



- score: 0.8311
- metadata: WeaveDict({'source': '2023 Q2 AAPL.pdf', 'raw_tokens': 10826, 'file_type': 'pdf'})
- chunk_number: 26
- chunk_id: dcedbab758188c3a7480d2fae442d427
- doc_id: 892ac7f7702cd56f574dae211e89bfa9


Apple Inc. | Q2 2023 Form 10-Q | 15Products and Services Performance
The following table shows net sales by category for the three- and six-month periods ended April 1, 2023 and March 26, 2022 (dollars in millions):
Three Months Ended Six Months Ended
April 1,
2023March 26,
2022 ChangeApril 1,
2023March 26,
2022 Change
Net sales by category:
iPhone $ 51,334 $ 50,570 2 %$ 117,109 $ 122,198 (4)%
Mac 7,168 10,435 (31)% 14,903 21,287 (30)%
iPad 6,670 7,646 (13)% 16,066 14,894 8 %
Wearables, Home and Accessories 8,757 8,806 (1)% 22,239 23,507 (5)%
Services 20,907 19,821 5 % 41,673 39,337 6 %
Total net sales $ 94,836 $ 97,278 (3)%$ 211,990 $ 221,223 (4)%
iPhone
iPhone net sales were relatively flat during the second quarter of 2023 compared to the secon d quarter of 2022.Year-over-year iPhone net sales decreased
during the first six months of 2023 due primarily to lower net sales from the Company’ s new iPhone models launched in the fourth quarter of 2022.Mac
Mac net sales decreased during the second quarter and first six months of 2023 compared to the same periods in 2022 due primarily to lower net sales of
MacBook Pro.iPad
iPad net sales decreased during the second quarter of 2023 compared to the second quarter of 2022 due primarily to lower net sales of iPad Pro  and iPad Air.Year-over-year iPad net sales increased during the first six months of 2023 due primarily to higher net sales of iPad, partially offset by lower net sales of iPad
mini .Wearables, Home and Accessories
Wearables, Home and Accessories net sales were relatively flat during the second quarter of 2023 compared to the second quarter of 2022.

---



- score: 0.829
- metadata: WeaveDict({'source': '2023 Q2 AAPL.pdf', 'raw_tokens': 10826, 'file_type': 'pdf'})
- chunk_number: 25
- chunk_id: edcf32124b6d8e0c6c6e4cc143b055a3
- doc_id: 892ac7f7702cd56f574dae211e89bfa9


During the first six months of 2023, the Europe net sales decrease consis ted primarily of lower net sales of Mac, partially offset by higher
net sales of iPhone.Greater China
The weakness in the renminbi relative to the U.S. dollar had an unfavorable year-over-year impact on Greater China net sales during the second quarter and
first six months of 2023.During the second quarter and first six months of 2023, the Greater China net sales decrease consisted primarily of lower net sales of
iPhone and Mac.Japan
The weakness in the yen relative to the U.S. dollar had an unfavorable year-over-year impact on Japan net sales during the second quarter and first six months
of 2023.During the second quarter of 2023, the Japan net sales decrease consisted primarily of lower net sales of iPad, Services and iPhone.During the first six
months of 2023, the Japan net sales decrease consisted primarily of lower net sales of Services, W earables, Home and Accessories and Mac.Rest of Asia Pacific
The weakness in foreign currencies relative to the U.S. dollar had an unfavorable year-over-year impact on Rest of Asia Pacific net sales during the second
quarter and first six months of 2023.During the second quarter and first six months of 2023, the Rest of Asia Pacific net sales increase consisted primarily of
higher net sales of iPhone, partially of fset by lower net sales of Mac.

---



- score: 0.8263
- metadata: WeaveDict({'source': '2023 Q1 AAPL.pdf', 'raw_tokens': 19713, 'file_type': 'pdf'})
- chunk_number: 25
- chunk_id: 08b9afbc4602c0acf9a05d7ebed88bb0
- doc_id: a1c822cc02697e5675aaa83c47357247


Rest of Asia Pacific
Rest of Asia Pacific net sales decreased during the first quarter of 2023 compared to the same quarter in 2022 due to the weakness in foreign currencies relative
to the U.S. dollar .The weakness in foreign curre ncies contributed to lower net sales of iPhone and Mac, which was partially offset by higher net sales of Services
and iPad.Products and Services Performance
The following table shows net sales by category for the three months ended December 31, 2022 and December 25, 2021 (dollars in millions):
Three Months Ended
December 31,
2022December 25,
2021 Change
Net sales by category:
iPhone $ 65,775 $ 71,628 (8)%
Mac 7,735 10,852 (29)%
iPad 9,396 7,248 30 %
Wearables, Home and Accessories 13,482 14,701 (8)%
Services 20,766 19,516 6 %
Total net sales $ 117,154 $ 123,945 (5)%
(1)Products net sales include amortization of the deferred value of unspecified software upgrade rights, which are bundled in the sales price of the respective
product.(2)Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod mini and accessories.(3)Services net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other services.Services net sales also include
amortization of the deferred value of services bundled in the sales price of certain products.iPhone
iPhone net sales decreased during the first quarter of 2023 compared to the same quarter in 2022 due primarily to lower net sales from the Company’ s new
iPhone models launched in the fourth quarter of 2022.Mac
Mac net sales decreased during the first quarter of 2023 compared to the same quarter in 2022 due primarily to lower net sales of MacBook Pro .iPad
iPad net sales increased during the first quarter of 2023 compared to the same quarter in 2022 due primarily to higher net sales of iPad and iPad Air.

---



...

Evaluated 7 of 9 examples

Evaluated 8 of 9 examples

Evaluated 9 of 9 examples

Evaluation summary
{
    'response_scorer': {
        'levenshtein': {'mean': 0.42383276036565865},
        'rouge': {'mean': 0.22455481785426695},
        'correctness': {
            'correct': {'true_count': 3, 'true_fraction': 0.3333333333333333},
            'extras': {'label': {'mean': 0.3333333333333333}, 'score': {'mean': 0.5715938210487366}}
        },
        'helpfulness': {
            'helpful': {'true_count': 7, 'true_fraction': 0.7777777777777778},
            'extras': {'label': {'mean': 0.7777777777777778}, 'score': {'mean': 0.8763572573661804}}
        },
        'bleu': {'mean': 0.07777881029510328},
        'relevance': {
            'extras': {'label': {'mean': 0.7777777777777778}, 'score': {'mean': 0.8035142289267646}},
            'relevant': {'true_count': 7, 'true_fraction': 0.7777777777777778}
        },
        'diff': {'mean': 0.06045990605326787},
        'meteor': {'mean': 0.23565555555555553}
    },
    'model_latency': {'mean': 9.077929284837511}
}

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fbf-c860-7c70-a107-b3b33e11dd13


### Reranking the contexts

What if we can retrive documents ? i.e we cast a wider net and retrieve more documents to improve the recall of the retrieval step.

However, with more contexts it is important to pick the ones that adds the most knowledge about the given query into the LLM context. i.e. we need to improve the precision of the retrieval step.

![](./imgs/Reranking.png)

For this, a re-ranking model (a separate machine learning model) can be used to calculate a matching score for a given query and document pair.
This score can then be used to rearrange vector search results, ensuring that the most relevant results are prioritized at the top of the list.
Cohere comes with it's own re-ranking model and is quite popular.


The `DenseRerankedRetriever` class below uses a re-ranking model over dense search engine to retrieve the most relevant documents and then re-rank them.


In [33]:
# Essentially the search engine remains the same but the retrieved documents are re-ranked using a re-ranking model.

class DenseRerankedRetriever(RetrieverWithReranker):
    pass

dense_reranked_retriever = DenseRerankedRetriever(search_engine=dense_search_engine,)


retrieved_docs = await dense_reranked_retriever.invoke(query=query, top_k=5)
for doc in retrieved_docs:
    render_doc(doc)

# Evaluate and compare again with the reranked pipeline.

retrieval_scores = await retrieval_evaluation.evaluate(
    model=dense_reranked_retriever,
    __weave={"display_name": "Dense Reranked Retrieval"}
)

class DenseRerankedRAGPipeline(SimpleRAGPipeline):
    pass

dense_reranked_rag_pipeline = DenseRerankedRAGPipeline(
    retriever=dense_reranked_retriever,
    generator=simple_response_generator
)

response_scores = await response_evaluation.evaluate(
    model=dense_reranked_rag_pipeline,
    __weave={"display_name": "Dense Reranked RAG Pipeline"}
)


🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fc0-0e75-7c03-98e0-965ba07837a4


- score: 0.6976
- metadata: WeaveDict({'source': '2023 Q2 AAPL.pdf', 'raw_tokens': 10826, 'file_type': 'pdf'})
- chunk_number: 26
- chunk_id: dcedbab758188c3a7480d2fae442d427
- doc_id: 892ac7f7702cd56f574dae211e89bfa9


Apple Inc. | Q2 2023 Form 10-Q | 15Products and Services Performance
The following table shows net sales by category for the three- and six-month periods ended April 1, 2023 and March 26, 2022 (dollars in millions):
Three Months Ended Six Months Ended
April 1,
2023March 26,
2022 ChangeApril 1,
2023March 26,
2022 Change
Net sales by category:
iPhone $ 51,334 $ 50,570 2 %$ 117,109 $ 122,198 (4)%
Mac 7,168 10,435 (31)% 14,903 21,287 (30)%
iPad 6,670 7,646 (13)% 16,066 14,894 8 %
Wearables, Home and Accessories 8,757 8,806 (1)% 22,239 23,507 (5)%
Services 20,907 19,821 5 % 41,673 39,337 6 %
Total net sales $ 94,836 $ 97,278 (3)%$ 211,990 $ 221,223 (4)%
iPhone
iPhone net sales were relatively flat during the second quarter of 2023 compared to the secon d quarter of 2022.Year-over-year iPhone net sales decreased
during the first six months of 2023 due primarily to lower net sales from the Company’ s new iPhone models launched in the fourth quarter of 2022.Mac
Mac net sales decreased during the second quarter and first six months of 2023 compared to the same periods in 2022 due primarily to lower net sales of
MacBook Pro.iPad
iPad net sales decreased during the second quarter of 2023 compared to the second quarter of 2022 due primarily to lower net sales of iPad Pro  and iPad Air.Year-over-year iPad net sales increased during the first six months of 2023 due primarily to higher net sales of iPad, partially offset by lower net sales of iPad
mini .Wearables, Home and Accessories
Wearables, Home and Accessories net sales were relatively flat during the second quarter of 2023 compared to the second quarter of 2022.

---



- score: 0.6518
- metadata: WeaveDict({'source': '2023 Q3 AAPL.pdf', 'raw_tokens': 10898, 'file_type': 'pdf'})
- chunk_number: 27
- chunk_id: d4bde8367402ec76fb4ebd18b5078d6d
- doc_id: a4d28158d90e45af01c17fa1bf5fe6f9


Apple Inc. | Q3 2023 Form 10-Q | 15Products and Services Performance
The following table shows net sales by category for the three- and nine-month periods ended July 1, 2023 and June 25, 2022 (dollars in millions):
Three Months Ended Nine Months Ended
July 1,
2023June 25,
2022 ChangeJuly 1,
2023June 25,
2022 Change
Net sales by category:
iPhone $ 39,669 $ 40,665 (2)%$ 156,778 $ 162,863 (4)%
Mac 6,840 7,382 (7)% 21,743 28,669 (24)%
iPad 5,791 7,224 (20)% 21,857 22,118 (1)%
Wearables, Home and Accessories 8,284 8,084 2 % 30,523 31,591 (3)%
Services 21,213 19,604 8 % 62,886 58,941 7 %
Total net sales $ 81,797 $ 82,959 (1)%$ 293,787 $ 304,182 (3)%
iPhone
iPhone net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales from
certain iPhone models, partially of fset by higher net sales of iPhone 14 Pro models.Mac
Mac net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales of laptops.iPad
iPad net sales decreased during the third quarter of 2023 compared to the third quarter of 2022 due primarily to lower net sales across most iPad models.Year-
over-year iPad net sales were relatively flat during the first nine months of 2023.Wearables, Home and Accessories
Wearables, Home and Accessories net sales increased during the third quarter of 2023 compare d to the third quarter of 2022 due primarily to higher net sales of
Wearables, which includes AirPods , Apple Watch  and Beats  products, partially offset by lower net sales of accessories.

---



- score: 0.6391
- metadata: WeaveDict({'source': '2022 Q3 AAPL.pdf', 'raw_tokens': 11081, 'file_type': 'pdf'})
- chunk_number: 25
- chunk_id: fed3877be83a64ed1694fddbcf141f7c
- doc_id: d526fb0c9708ec6e7103a267f08b9a6a


Furth er informat ion regarding the Com pany’ s reportable segments can be found in Part I, Item 1 of this Form 10-Q in the Notes to Condensed
Consolidated Financial Statements in Note 9, “Segment Information and Geographic Data.”
The following table shows net sales by reportable segment for the three- and nine-month periods ended June 25, 2022 and June 26, 2021 (dollars in millions):
Three Months Ended Nine Months Ended
June 25,
2022June 26,
2021 ChangeJune 25,
2022June 26,
2021 Change
Net sales by reportable segment:
Americas $ 37,472 $ 35,870 4 %$ 129,850 $ 116,486 11 %
Europe 19,287 18,943 2 % 72,323 68,513 6 %
Greater China 14,604 14,762 (1)% 58,730 53,803 9 %
Japan 5,446 6,464 (16)% 20,277 22,491 (10)%
Rest of Asia Pacific 6,150 5,395 14 % 23,002 21,164 9 %
Total net sales $ 82,959 $ 81,434 2 %$ 304,182 $ 282,457 8 %
Americas
Americas net sales increased during the third quarter of 2022 compared to the third quarter of 2021 due primarily to higher net sales of Services and iPhone,
partially offset by lower net sales of Mac and Wearables, Home and Accessories.Year-over-ye ar Americas net sales increased during the first nine months of
2022 due primarily to higher net sales of iPhone, Services and W earables, Home and Accessories.Europe
Europe net sales increas ed during the third quarter of 2022 compared to the third quarter of 2021 due primarily to higher net sales of iPhone, partially offset by
lower net sales of Mac.Year-over-year Europe net sales increased during the first nine months of 2022 due primarily to higher net sales of Services and iPhone.The weakness in foreign  currencies relative to the U.S. dollar had a net unfavorable impact on Europe net sales during the third quarter and first nine months of
2022.

---



- score: 0.6043
- metadata: WeaveDict({'source': '2023 Q2 AAPL.pdf', 'raw_tokens': 10826, 'file_type': 'pdf'})
- chunk_number: 24
- chunk_id: 7327384e8668320d09f2e673d772545a
- doc_id: 892ac7f7702cd56f574dae211e89bfa9


The Company repurchased $19.1 billion of its common stock and paid dividends and dividend equivalents of $3.7 billion during the second quarter of 2023.®
®
®
Apple Inc. | Q2 2023 Form 10-Q | 14Macroeconomic Conditions
Macroeconomic conditions, including inflation, changes in interest rates, and currency fluctuations, have directly and indirectly impacted, and could in the future
materially impact, the Company’ s results of operations and financial condition.Segment Operating Performance
The following table shows net sales by reportable segment for the three- and six-month periods ended April 1, 2023 and March 26, 2022 (dollars in millions):
Three Months Ended Six Months Ended
April 1,
2023March 26,
2022 ChangeApril 1,
2023March 26,
2022 Change
Net sales by reportable segment:
Americas $ 37,784 $ 40,882 (8)%$ 87,062 $ 92,378 (6)%
Europe 23,945 23,287 3 % 51,626 53,036 (3)%
Greater China 17,812 18,343 (3)% 41,717 44,126 (5)%
Japan 7,176 7,724 (7)% 13,931 14,831 (6)%
Rest of Asia Pacific 8,119 7,042 15 % 17,654 16,852 5 %
Total net sales $ 94,836 $ 97,278 (3)%$ 211,990 $ 221,223 (4)%
Americas
Americas net sales decreased during the second  quarter and first six months of 2023 compared to the same periods in 2022 due primarily to lower net sales of
iPhone and Mac, partially of fset by higher net sales of Services.Europe
The weakness in foreign  currencies relative to the U.S. dollar had a net unfavorable year-over-y ear impact on Europe net sales during the second quarter and
first six months of 2023.During the second quarter of 2023, the Europe net sales increase consisted primarily of higher net sales of iPhone, partially offset by
lower net sales of Mac.

---



...

Evaluated 7 of 9 examples

Evaluated 8 of 9 examples

Evaluated 9 of 9 examples

Evaluation summary
{
    'response_scorer': {
        'levenshtein': {'mean': 0.43245938147741814},
        'rouge': {'mean': 0.22035437105373934},
        'correctness': {
            'correct': {'true_count': 4, 'true_fraction': 0.4444444444444444},
            'extras': {'label': {'mean': 0.4444444444444444}, 'score': {'mean': 0.630999743938446}}
        },
        'helpfulness': {
            'helpful': {'true_count': 7, 'true_fraction': 0.7777777777777778},
            'extras': {'label': {'mean': 0.7777777777777778}, 'score': {'mean': 0.875082532564799}}
        },
        'bleu': {'mean': 0.08692802905100354},
        'relevance': {
            'extras': {'label': {'mean': 0.6666666666666666}, 'score': {'mean': 0.7353867689768473}},
            'relevant': {'true_count': 6, 'true_fraction': 0.6666666666666666}
        },
        'diff': {'mean': 0.06487963488494794},
        'meteor': {'mean': 0.23995555555555553}
    },
    'model_latency': {'mean': 12.067407899432713}
}

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fc0-64a6-7201-bd64-556a08308591


### Hybrid Retrieveal

Even though BM25 is an old model used for retrieval tasks, it is still the state-of-the-art on various benchmark. In machine learning, we ensemble a few weak classifiers to build a stronger classifier, we can adopt the same idea to our retriever pipeline.

Here we show the concept of hybrid retriever which uses two or more retrievers and retrieves chunks from all of them followed by re-ranking.

![](./imgs/HybridRetrieval.png)

We use both the BM25 and Dense retrieval algorithms to retrieve chunks from both of them and then re-rank them.

One additional thing to note here is to deduplicate the chunks retrieved from both the retrievers before re-ranking. This is because repeated chunks are not useful and can lead to a worse performance if not deduplicated.

In [34]:
# Setup the hybrid retriever with the sparse and dense search engines.
hybrid_retriever = HybridRetrieverWithReranker(
    sparse_search_engine=bm25_search_engine,
    dense_search_engine=dense_search_engine
)

retrieved_docs = await hybrid_retriever.invoke(query=query, top_k=10, top_n=5)

for doc in retrieved_docs:
    render_doc(doc)


hybrid_retrieval_scores = await retrieval_evaluation.evaluate(
    model=hybrid_retriever,
    __weave={"display_name": "Hybrid Retrieval"}
)

class HybridRAGPipeline(SimpleRAGPipeline):
    pass

hybrid_rag_pipeline = HybridRAGPipeline(
    retriever=hybrid_retriever,
    generator=simple_response_generator
)

hybrid_response_scores = await response_evaluation.evaluate(
    model=hybrid_rag_pipeline,
    __weave={"display_name": "Hybrid RAG Pipeline"}
)



🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fc0-bb3c-7770-bd1a-2a21ae0a3355


- score: 0.7216
- metadata: WeaveDict({'source': '2022 Q3 AAPL.pdf', 'raw_tokens': 11081, 'file_type': 'pdf'})
- chunk_number: 21
- chunk_id: b2d7313fb7914541a5f775519a2a3fe5
- doc_id: d526fb0c9708ec6e7103a267f08b9a6a


Available Information
The Company periodically provides certain information for investors on its corporate website, www .apple.com, and its investor relations website,
investor .apple.com.This includes press releases and other information about financial performance, information on environmental, social and corporate
governance matters, and details related to the Company’ s annual meeting of shareholders.The information contained on the websites referenced in this Form
10-Q is not incorporated by reference into this filing.Further , the Company’ s references to website URLs are intended to be inactive textual references only .Business Seasonality and Product Introductions
The Company has historically experienced higher net sales in its first quarter compared to other quarters in its fiscal year due in part to seasonal holiday
demand.Additionally , new product and service introductions can significantly impact net sales , cost of sales and operating expenses.The timing of product
introductions can also impact the Company’ s net sales to its indirect distribution channels as these channels are filled with new inventory following a product
launch, and channel inventory of an older product often declines as the launch of a newer produ ct approaches.Net sales can also be affected when consumers
and distributors anticipate a product introduction.Quarterly Highlights
Total net sales increased 2% or $1.5 billion during the third quarter of 2022 compared to the same quarter in 2021, driven primarily by higher net sales of
Services and iPhone, partially offset by lower net sales of Mac and Wearables, Home and Accessories.The weakness in foreign currencies relative to the U.S.
dollar had an unfavorable impact on all Products and Services net sales during the third quarter of 2022.At the end of the third quarter of 2022, the Com pany introduced an all-new MacBook Air and an updated 13-inch  MacBook Pro , both powe red by the new
Apple M2 chip.The Company also announced iOS 16, macOS  Ventura, iPadOS  16 and watchOS  9, updates to its operating systems that are expe cted to be
available in the fall of 2022.

---



- score: 0.7012
- metadata: WeaveDict({'source': '2023 Q2 AAPL.pdf', 'raw_tokens': 10826, 'file_type': 'pdf'})
- chunk_number: 26
- chunk_id: dcedbab758188c3a7480d2fae442d427
- doc_id: 892ac7f7702cd56f574dae211e89bfa9


Apple Inc. | Q2 2023 Form 10-Q | 15Products and Services Performance
The following table shows net sales by category for the three- and six-month periods ended April 1, 2023 and March 26, 2022 (dollars in millions):
Three Months Ended Six Months Ended
April 1,
2023March 26,
2022 ChangeApril 1,
2023March 26,
2022 Change
Net sales by category:
iPhone $ 51,334 $ 50,570 2 %$ 117,109 $ 122,198 (4)%
Mac 7,168 10,435 (31)% 14,903 21,287 (30)%
iPad 6,670 7,646 (13)% 16,066 14,894 8 %
Wearables, Home and Accessories 8,757 8,806 (1)% 22,239 23,507 (5)%
Services 20,907 19,821 5 % 41,673 39,337 6 %
Total net sales $ 94,836 $ 97,278 (3)%$ 211,990 $ 221,223 (4)%
iPhone
iPhone net sales were relatively flat during the second quarter of 2023 compared to the secon d quarter of 2022.Year-over-year iPhone net sales decreased
during the first six months of 2023 due primarily to lower net sales from the Company’ s new iPhone models launched in the fourth quarter of 2022.Mac
Mac net sales decreased during the second quarter and first six months of 2023 compared to the same periods in 2022 due primarily to lower net sales of
MacBook Pro.iPad
iPad net sales decreased during the second quarter of 2023 compared to the second quarter of 2022 due primarily to lower net sales of iPad Pro  and iPad Air.Year-over-year iPad net sales increased during the first six months of 2023 due primarily to higher net sales of iPad, partially offset by lower net sales of iPad
mini .Wearables, Home and Accessories
Wearables, Home and Accessories net sales were relatively flat during the second quarter of 2023 compared to the second quarter of 2022.

---



- score: 0.6783
- metadata: WeaveDict({'source': '2023 Q1 AAPL.pdf', 'raw_tokens': 19713, 'file_type': 'pdf'})
- chunk_number: 22
- chunk_id: befc7b2d0770ac61076f9121f259d82b
- doc_id: a1c822cc02697e5675aaa83c47357247


Additionally , new product and service introductions can significantly impact net sales , cost of sales and operating expenses.The timing of product
introductions can also impact the Company’ s net sales to its indirect distribution channels as these channels are filled with new inventory following a product
launch, and channel inventory of an older product often declines as the launch of a newer produ ct approaches.Net sales can also be affected when consumers
and distributors anticipate a product introduction.Fiscal Period
The Compan y’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September .An additional week is included in the first fiscal quarter
every five or six years to realign the Company’ s fiscal quarters with calendar quarters, which occurred in the first quarter of 2023.The Company’ s fiscal years
2023 and 2022 span 53 and 52 weeks, respectively .Quarterly Highlights
Total net sales decrease d 5% or $6.8 billion during the first quarter of 2023 compared to the same quarter in 2022 due to the weakness in foreign currencies
relative to the U.S. dolla r. The weakness in foreign currencies contributed to lower net sales of iPhone and Mac, which was partially offset by higher net sales of
iPad.During the first quarter of 2023, the Company announced a new iPad, a new iPad Pro  powered by the Apple M2 chip, and a new Apple TV 4K.The Company repurchased $19.0 billion of its common stock and paid dividends and dividend equivalents of $3.8 billion during the first quarter of 2023.®
Apple Inc. | Q1 2023 Form 10-Q | 15COVID-19
The COVID-19 pandemic has had, and continue s to have, a significant impact around the world, prompting governments and businesses to take unprecedented
measures, such as restrictions on travel and business operations, temporary closures of busines ses, and quarantine and shelter-in-place orders.The COVID-19
pandemic has at times significantly curtailed global economic activity and caused significant volatility and disruption in global financial markets.The COVID-19
pandemic and the meas ures taken by many countries in response have affected and could in the future materially impact the Company’ s business, results of
operations and financial condition.

---



- score: 0.6447
- metadata: WeaveDict({'source': '2023 Q3 AAPL.pdf', 'raw_tokens': 10898, 'file_type': 'pdf'})
- chunk_number: 27
- chunk_id: d4bde8367402ec76fb4ebd18b5078d6d
- doc_id: a4d28158d90e45af01c17fa1bf5fe6f9


Apple Inc. | Q3 2023 Form 10-Q | 15Products and Services Performance
The following table shows net sales by category for the three- and nine-month periods ended July 1, 2023 and June 25, 2022 (dollars in millions):
Three Months Ended Nine Months Ended
July 1,
2023June 25,
2022 ChangeJuly 1,
2023June 25,
2022 Change
Net sales by category:
iPhone $ 39,669 $ 40,665 (2)%$ 156,778 $ 162,863 (4)%
Mac 6,840 7,382 (7)% 21,743 28,669 (24)%
iPad 5,791 7,224 (20)% 21,857 22,118 (1)%
Wearables, Home and Accessories 8,284 8,084 2 % 30,523 31,591 (3)%
Services 21,213 19,604 8 % 62,886 58,941 7 %
Total net sales $ 81,797 $ 82,959 (1)%$ 293,787 $ 304,182 (3)%
iPhone
iPhone net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales from
certain iPhone models, partially of fset by higher net sales of iPhone 14 Pro models.Mac
Mac net sales decreased during the third quarter and first nine months of 2023 compared to the same periods in 2022 due primarily to lower net sales of laptops.iPad
iPad net sales decreased during the third quarter of 2023 compared to the third quarter of 2022 due primarily to lower net sales across most iPad models.Year-
over-year iPad net sales were relatively flat during the first nine months of 2023.Wearables, Home and Accessories
Wearables, Home and Accessories net sales increased during the third quarter of 2023 compare d to the third quarter of 2022 due primarily to higher net sales of
Wearables, which includes AirPods , Apple Watch  and Beats  products, partially offset by lower net sales of accessories.

---



...

Evaluated 7 of 9 examples

Evaluated 8 of 9 examples

Evaluated 9 of 9 examples

Evaluation summary
{
    'retrieval_scorer': {
        'precision': {'mean': 0.5851851851851851},
        'hit_rate': {'mean': 0.5777777777777778},
        'map': {'mean': 0.7611111111111111},
        'relevance': {'mean': 0.0},
        'relevance_score': {'mean': 0.35927777777777775},
        'mrr': {'mean': 0.2659259259259259},
        'ndcg': {'mean': 0.573722800371186},
        'f1_score': {'mean': 0.5938271604938272},
        'recall': {'mean': 0.6666666666666666}
    },
    'model_latency': {'mean': 1.7241240077548556}
}

🍩 https://wandb.ai/a-sh0ts/rag-workshop-pc-nyc-3/r/call/01948fc0-c091-7441-9b9b-c0d97cf6dec2


## Can we do better ?

What if we have more data ? We initially used only the W&B documentation for our RAG pipeline. What if we use more data sources, like the `Weave Documentation`, `W&B Tutorials` and `Examples`, `W&B Blog posts`, `W&B Course content` etc. 

While it might improve the performance of the RAG pipeline, it might also increase the cost and complexity the pipeline. 
Our naive `DenseSearchEngine` might not be able to handle such a large corpus of documents because it requires storing the embeddings in memory.

One specific way to handle some of these issues is to use a vector database to store the embeddings of the documents and then use a retriever to retrieve the most relevant documents.
Vector databases are optimized for storing and querying embeddings and are a good way to handle large corpora of documents. 
This allows us to work with a large corpus of documents without dealing with the complexity of storing the embeddings in memory.

We will be using the `VectorStoreSearchEngine` class to create a vector store search engine.

In [35]:
from utils import prepare_documents_for_pinecone

In [36]:
pinecone_docs = prepare_documents_for_pinecone(document_chunks)

In [39]:
vectorstore_search_engine = VectorStoreSearchEngine()
if STORE_INTO_PINECONE:
    vectorstore_search_engine = await vectorstore_search_engine.fit(pinecone_docs)
else:
    vectorstore_search_engine = await vectorstore_search_engine.load()

One additional advantage of using a vector database is we can run more complex queries that include metadata filters.

For instance, we can retrieve all the markdown and notebook files from the dataset without retrieving the code documents. This can be useful if we are interested in retrieving a specific type of document or subset of documents before preforming the retrieval.

In [ ]:
results = await vectorstore_search_engine.search(
    query=query,
    top_k=5,
    filters={"file_type": {"$in": ["pdf"]}}
)
for doc in results:
    render_doc(doc)

In [ ]:
results[0]

In [ ]:
class VectorStoreRetriever(RetrieverWithReranker):
    pass

vectorstore_retriever = VectorStoreRetriever(search_engine=vectorstore_search_engine)

#TODO: Appropriate filters
results = await vectorstore_retriever.invoke(query=query, top_k=10, top_n=5, filters={"file_type": {"$in": ["pdf"]}})
for doc in results:
    render_doc(doc)

class VectorStoreRAGPipeline(SimpleRAGPipeline):
    pass

vectorstore_rag_pipeline = VectorStoreRAGPipeline(
    retriever=vectorstore_retriever,
    generator=simple_response_generator)

response = await vectorstore_rag_pipeline.invoke(query=query)

printmd(response["answer"])

# We can now evaluate the response generation performance of the vector store RAG pipeline.
vectorstore_response_scores = await response_evaluation.evaluate(
    model=vectorstore_rag_pipeline,
    __weave={"display_name": "Vector Store RAG Pipeline"}
)
# Compare the scores of the vector store RAG pipeline with the earlier pipelines.

## Query Enhancement

Next let's look at another way to improve our RAG pipeline.
We will be implementing a *query enhancement* stage in out pipeline.  Our `QueryEnhancer` will perform two key tasks:


![QueryEnhancer](./imgs/AdvancedRAG.png)

1. **Intent Classification**: Determine the user's intent of based on the query. This helps us better understand the user's query and improve the response generation step.

2. **Query Expansion**: Break down queries into more focused sub-queries for retrieval. This improves retrieval by capturing different aspects of the original question.

These enhancements serve two primary purposes:
- Inform the response generator, allowing it to tailor its output based on language and intent.
- Improve the retrieval process by using more targeted sub-queries.

Let's implement our `QueryEnhancer` and see it in action:

In [ ]:
# Again the `QueryEnhancer` is a `weave.Model`
# it contains two methods, one to get the intent prediction and one to generate search queries.
# the invoke method is a wrapper around the two methods.
# let's see how it works for a sample query.
query_enhancer = QueryEnhancer()
results = await query_enhancer.invoke(query=query)
results


We can now use the `QueryEnhancer` to create a `QueryEnhancedRAGPipeline`. However, unlike the `SimpleRAGPipeline`, this pipeline requires a `response_generator` that can handle the user's intent and query. Additionally, the `retriever` must now be able to handle the search queries generated by the `QueryEnhancer` and rerank and collate the results.

The `QueryEnhancedRAGPipeline` encapsulates this logic and provides a way to unify the pipeline with the `QueryEnhancer` and the response generator.

In [ ]:
# Instantiate the `QueryEnhancedRAGPipeline` with the `QueryEnhancer`, `retriever` and `QueryEnhancedResponseGenerator`.
query_enhanced_rag_pipeline = QueryEnhancedRAGPipeline(
    query_enhancer=query_enhancer,
    retriever=vectorstore_retriever,
    generator=QueryEnhancedResponseGenerator()
)
# We can now evaluate the response generation performance of the query enhanced RAG pipeline.
response = await query_enhanced_rag_pipeline.invoke(query=query)
printmd(response["answer"])

query_enhanced_response_scores = await response_evaluation.evaluate(
    model=query_enhanced_rag_pipeline,
    __weave={"display_name": "Query Enhanced RAG Pipeline"}
)

## Agentic RAG

So far we have seen a few ways to improve our RAG pipeline. 
The query enhanced RAG pipeline is already a little agentic in the sense that it uses the `QueryEnhancer` to generate search queries and then uses the `retriever` to retrieve the most relevant documents. The query enhancer uses a LLM to generate the search queries and intents predictions. We then manually route these to the retriever and response generator. It also made use of Structured Outputs to return the results in a structured format. - This was an example of an llm using tool calling. 

Tool use allows for greater flexibility in accessing and utilizing data sources, thus unlocking new use cases not possible with a standard RAG approach.

In a setting where data sources are diverse with non-homogeneous formats (structured/semi-structured/unstructured), this approach becomes even more important.

We'll look at how we can implement an agentic RAG system using a tool use approach. The agent can search for information about how to use the product, retrieve information from the internet, and search code examples

Concretely, we'll cover the following use cases:

- `search_docs`: Searches the filing documents in the SEC database
- `search_internet`: Searches the internet for general queries

With tool use we can take out pipeline a step further and create an agentic system.
We will be using the `Agent` class to create an agentic RAG pipeline. 

The `Agent` class is a `weave.Model` that encapsulates some tools and llm calls and their routing logic. Let's see how it works.



In [45]:
from agent import Agent

In [ ]:
agent = Agent()

answer = await agent.invoke(query="Compare the latest APPL filing to the TSLA filing in our database")
printmd(answer["answer"])
